# import mudules

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import pymc3 as pm
import seaborn as sns
import theano.tensor as tt
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from itertools import permutations

# hyper parameters

In [ ]:
K = 3  # num of components
D = 2  # dimensions
SEED = 1234
PERCENT = 0.1
DATA_PATH = './UAH/'

# utility functions

In [ ]:
def sampling(w, mu, sigma, N=2000):
    num = int(w * N)
    if len(mu) == 1:
        samples = np.random.normal(mu, sigma, size=num)
    else:
        samples = np.random.multivariate_normal(mu, sigma, size=num)
    return samples

def make_cov_matrix(chol, module=np):
    C = module.zeros((D,D))
    idxl = np.mask_indices(D, np.tril)
    if module == np:
        C[idxl] = chol
    else:
        C = tt.set_subtensor(C[idxl], chol)
    cov = C.dot(C.T)
    return cov

def kalman(obs, x, p=np.eye(2), dt=0.1):
    eta = []
    for i in range(len(obs)):
        A = np.array([[1, dt], [0, 1]])
        C = np.array([[1, 0]])
        Q = 100 * np.eye(2)
        R = 1
        x = A.dot(x)
        p = A.dot(p).dot(A.T) + Q
        k = p.dot(C.T).dot(np.linalg.inv(C.dot(p).dot(C.T) + R))
        x = x + k.dot(obs[i] - C.dot(x))
        p = (np.eye(2) - k.dot(C)).dot(p)
        eta.append(x[1])
    return np.array(eta)

def outlier(s, x, level=1):
    mean_x = np.mean(s)
    std_x = np.std(s)
    index = np.where(np.abs(s - mean_x) >= level * std_x)
    x = [np.delete(x[k], index) for k in range(len(x))]
    return x

def grad(t, x, m):
    headway = []
    speed = []
    maneuver = []
    i_0 = []
    i_f = []
    for i in range(len(x) - 1):
        if x[i] == -1 and x[i + 1] != -1:
            i_0.append(i)
        if x[i] != -1 and x[i + 1] == -1:
            i_f.append(i)
    num = np.min([len(i_0), len(i_f)])
    for i in range(num):
        if len(x[i_0[i] + 1: i_f[i] + 1]) < 2:
            continue
        s = x[i_0[i] + 1: i_f[i] + 1]
        ds1 = np.gradient(x[i_0[i] + 1: i_f[i] + 1], t[i_0[i] + 1: i_f[i] + 1])
        ds2 = kalman(s, np.array([s[0], ds1[0]]))
        headway.append(s)
        speed.append(ds2)
        maneuver.append(m[i_0[i] + 1: i_f[i] + 1])
    headway = np.concatenate(headway)
    speed = np.concatenate(speed)
    maneuver = np.concatenate(maneuver)
    headway, speed, maneuver = outlier(headway, (headway, speed, maneuver), 1)
    headway, speed, maneuver = outlier(speed, (headway, speed, maneuver), 1)
    return headway, speed, maneuver

def norm1d_plot(x):
    mu = x.mean()
    sigma = x.std()
    v = np.linspace(x.min(), x.max(), 50)
    s = 1 / (np.sqrt(2 * 3.14) * sigma) * np.exp(-(v - mu) ** 2 / (2 * sigma ** 2))
    plt.plot(v, s)
    return s.max()

def ordered(results, gmm):
    order = list(permutations(range(K)))
    mse = np.zeros(len(order))
    num = 0
    for l in order:
        w = [gmm.weights_[k] for k in l]
        mu = [gmm.means_[k] for k in l]
        L = [np.linalg.cholesky(gmm.covariances_[k]) for k in l]
        ixl = np.tril_indices(D)
        chol = [L[k][ixl] for k in range(K)]
        gmm_vars = [[w], mu, chol]
        value = 0.
        for n in range(3):
            for (var1, var2) in zip(results[n], gmm_vars[n]):
                col = var1.shape[1]
                value += np.sum([(var1[:, k].mean() - var2[k]) ** 2 for k in range(col)])
        mse[num] = value
        num += 1
    index = mse.argmin()
    w = [gmm.weights_[k] for k in order[index]]
    mu = [gmm.means_[k] for k in order[index]]
    L = [np.linalg.cholesky(gmm.covariances_[k]) for k in order[index]]
    chol = [np.array([L[k][0, 0], L[k][1, 0], L[k][1, 1]]) for k in range(K)]
    gmm_vars = [[w], mu, chol]
    return gmm_vars

def prior_model(x, w, mu, cov):
    prob = []
    for v in x:
        p = 0.
        for i in range(K):
            c = np.sqrt(1 / ((2 * 3.14) ** D * np.linalg.det(cov[k])))
            f = c * np.exp(-0.5 * (v - mu[k]).dot(np.linalg.inv(cov[k])).dot((v - mu[k]).T))
            p += w[k] * f
        prob.append(p)
    return np.array(prob)

# load data

In [ ]:
header_1 = ['time', 'headway', 'ttc', 'num', 'speed']
header_2 = ['time', 'maneuver', 'lat', 'lon', 'duration', 'th']
time = []
hs = []
manu = []
for root, dirs, file in os.walk(DATA_PATH+'D2/'):
    for path in dirs:
        detection = pd.read_csv(os.path.join(root, path, 'PROC_VEHICLE_DETECTION.txt'), sep='\ ', names=header_1)
        event = pd.read_csv(os.path.join(root, path, 'EVENTS_LIST_LANE_CHANGES.txt'), sep='\ ', names=header_2)
        time_1 = np.around(detection['time'].values, 1)
        time_2 = np.around(event['time'].values, 1)
        headway = detection['headway'].values
        t = np.arange(time_1[0], time_1[-1], 0.1)
        s = np.interp(t, time_1, headway)
        index = []
        for d in time_2:
            index.append(np.where(np.abs(t - d) <= 1e-3)[0][0])
        m = np.zeros_like(t)
        m[index] = np.sign(event['maneuver'].values)
        time.append(t)
        hs.append(s)
        manu.append(m)
time = np.concatenate(time)
hs = np.concatenate(hs)
manu = np.concatenate(manu)
x0, dx0, m0 = grad(time, hs, manu)

In [ ]:
num = int(len(x0) * PERCENT)
samples = np.random.choice(len(x0), num, replace=False)
x = x0[samples]
dx = dx0[samples]
m = m0[samples]
data = pd.DataFrame([x, dx]).transpose()
data.columns = ['x', 'dx']
data = data.dropna(axis=0)

In [ ]:
plt.figure()
data['x'].plot()
plt.figure()
data['dx'].plot()
plt.figure()
sns.heatmap(data.corr(), annot=True)

# modeling

In [ ]:
with pm.Model() as model:
    if K == 1:
        if D == 1:
            mu = pm.Normal('mu0', mu=20, sd=5, shape=D)
            sigma = pm.Gamma('sigma0', alpha=25, beta=5, shape=D)
            obs = pm.Normal('obs', mu=mu, sd=sigma, observed=data['x'])
        else:
            mu = pm.Normal('mu0', mu=[20, 0], sd=[5, 5], shape=D)
            tril = pm.LKJCholeskyCov('chol0', n=D, eta=1, sd_dist=pm.HalfCauchy.dist(2.5))
            chol = pm.expand_packed_triangular(D, tril)
            obs = pm.MvNormal('obs', mu=mu, chol=chol, observed=data)
    else:
        if D == 1:
            w = pm.Dirichlet('w', a=np.ones(K))
            mu = tt.stack([pm.Normal('mu'+str(k), mu=10, sd=10, shape=D) for k in range(K)])
            sigma = tt.stack([pm.Gamma('sigma'+str(k), alpha=25, beta=5, shape=D) for k in range(K)])
            dist = [pm.Normal.dist(mu=mu[k], sd=sigma[k]) for k in range(K)]
            obs = pm.Mixture('obs', w=w, comp_dists=dist, observed=data['x'])
        else:
            w = pm.Dirichlet('w', a=np.ones(K))
            mu = tt.stack([pm.Normal('mu'+str(k), mu=10, sd=10, shape=D) for k in range(K)])
            tril = tt.stack([pm.LKJCholeskyCov('chol'+str(k), n=D, eta=1., sd_dist=pm.HalfCauchy.dist(2.5)) for k in range(K)])
            chol = tt.stack([pm.expand_packed_triangular(D, tril[k]) for k in range(K)])
            dist = [pm.MvNormal.dist(mu=mu[k], chol=chol[k]) for k in range(K)]
            obs = pm.Mixture('obs', w=w, comp_dists=dist, observed=data)

## traning

In [ ]:
# with model:
#     inference = pm.fit(method='advi')
#     trace = inference.sample()

In [ ]:
with model:
    trace = pm.sample(chains=1)

## results

In [ ]:
with model:
    pm.traceplot(trace)

In [ ]:
with model:
    pm.plot_posterior(trace)

In [ ]:
with model:
    pm.forestplot(trace)

In [ ]:
with model:
    pm.autocorrplot(trace)

In [ ]:
with model:
    pm.energyplot(trace)

In [ ]:
with model:
    pm.densityplot(trace)

In [ ]:
pm.summary(trace)

## joint distribution

In [ ]:
data_post = []
if D == 1:
    w_post = trace['w'].mean(0)
    mu_post = [trace['mu'+str(k)].mean(0) for k in range(K)]
    sigma_post = [trace['sigma'+str(k)].mean(0) for k in range(K)]
    for k in range(K):
        nxy = sampling(w=w_post[k], mu=mu_post[k], sigma=sigma_post[k])
        data_post.append(nxy)
    data_post = np.concatenate(data_post)
    sns.distplot(data['x'], bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Prior distribution')
    sns.distplot(data_post, bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Posterior distribution')
    plt.legend()
else:
    w_post = trace['w'].mean(0)
    mu_post = [trace['mu'+str(k)].mean(0) for k in range(K)]
    chol_post = [trace['chol'+str(k)].mean(0) for k in range(K)]
    cov_post = [make_cov_matrix(chol=chol_post[k]) for k in range(K)]
    plt.figure()
    plt.xlim([4, 45])
    plt.ylim([-20, 20])
    for k in range(K):
        nxy = sampling(w=w_post[k], mu=mu_post[k], sigma=cov_post[k], N=num)
        sns.scatterplot(nxy[:, 0], nxy[:, 1], label='component'+str(k))
        data_post.append(nxy)
    data_post = np.concatenate(data_post)
    data_post = pd.DataFrame(data_post)
    data_post.columns = ['x', 'dx']
    plt.figure()
    sns.jointplot('x', 'dx', data=data, xlim=[5, 45], ylim=[-20, 20], kind='kde', space=0, color='r')
    sns.jointplot('x', 'dx', data=data_post, xlim=[5, 45], ylim=[-20, 20], kind='kde', space=0, color='g')

## marginalized distribution

In [ ]:
plt.figure()
sns.distplot(data['x'], bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Prior distribution')
sns.distplot(data_post['x'], bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Posterior distribution')
plt.legend()
plt.figure()
sns.distplot(data['dx'], bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Prior distribution')
sns.distplot(data_post['dx'], bins=30, kde=False, norm_hist=True, hist_kws={'histtype': 'step', 'linewidth': 3}, label='Posterior distribution')
plt.legend()

In [ ]:
gmm = GaussianMixture(K)
gmm.fit(data)
p1 = prior_model(data.values, gmm.weights_, gmm.means_, gmm.covariances_)
p2 = prior_model(data.values, w_post, mu_post, cov_post)
p1 = p1 / np.sum(p1)
p2 = p2 / np.sum(p2)
plt.figure(figsize=[16, 8])
plt.subplot(121)
plt.scatter(data['x'], data['dx'], c=p1)
plt.colorbar()
plt.subplot(122)
plt.scatter(data['x'], data['dx'], c=p2)
plt.colorbar()

In [ ]:
w_results = [trace.get_values('w')]
mu_results = [trace.get_values('mu'+str(k)) for k in range(K)]
chol_results = [trace.get_values('chol'+str(k)) for k in range(K)]
results = [w_results, mu_results, chol_results]
gmm_vars = ordered(results, gmm)
title = ['w', 'mu', 'chol']
for n in range(3):
    plt.figure(figsize=[16, 8])
    i = 0
    for (var1, var2) in zip(results[n], gmm_vars[n]):
        col = var1.shape[1]
        plt.subplot2grid([len(results), 2], [i, 0])
        plt.ylabel('Frequency')
        plt.title(title[n]+str(i))
        ylim = [norm1d_plot(var) for var in var1.T]
        [plt.plot([var, var], [0, np.max(ylim)], color='r') for var in var2]
        plt.subplot2grid([len(results), 2], [i, 1])
        plt.plot(var1)
        plt.ylabel('Sample value')
        plt.title(title[n]+str(i))
        plt.subplots_adjust(wspace=0.2, hspace=0.5)
        i += 1

In [ ]:
N = 10000
index = np.random.choice(data.shape[0], N, True, p2)
samples = data.iloc[index]
s = m[index]
plt.scatter(samples['x'], samples['dx'])
plt.scatter(samples['x'].iloc[s != 0], samples['dx'].iloc[s != 0])
print(samples['x'].iloc[s != 0].shape[0] / N)